# COMP0005 - GROUP COURSEWORK 2023-24
# Gesture Recognition via Convex Hull 

Use the cell below for all python code needed to realise the **Jarvis march algorithm** (including auxiliary data structures and functions needed by this algorithm - if any). The `jarvismarch()` function itself should take as input parameter a list of 2D inputSet (`inputSet`), and return the subset of such inputSet (`outputSet`) that lie on the convex hull.

In [1]:
def jarvismarch(inputSet):      
    '''
    Returns the list of inputSet that lie on the convex hull (jarvis march algorithm)
            Parameters:
                    inputSet (list): a list of 2D inputSet

            Returns:
                    outputSet (list): a list of 2D inputSet
    '''

    leftMostPoint = findLeftMostPoint(inputSet)
    outputSet = []
        
    origin = inputSet[leftMostPoint]
    outputSet.append(origin)
        
    p = leftMostPoint
        
    while True:
        q = (p + 1) % len(inputSet)
        for i in range(len(inputSet)):
                if isBetterOrientation(inputSet[p], inputSet[q], inputSet[i]):
                        q = i
        p = q
        if p == leftMostPoint:
                break
                
        outputSet.append(inputSet[p])

    return outputSet

Use the cell below for all python code needed to realise the **Graham scan** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `grahamscan()` function itself should take as input parameter a list of 2D inputSet (`inputSet`), and return the subset of such inputSet that lie on the convex hull (`outputSet`).

In [2]:
class Point:
    def __init__(self,x,y):
        self[0] = x
        self[1] = y
    
    def calcDistance(self,point2):
        return abs(point2[0]-self[0]) + abs(point2[1]-self[1])

class PointSymbolTable: #technically a binary tree
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.left = None
        self.right = None
    
    def valuesToList(self):#in order traversal to list
        return (self.left.valuesToList() if self.left else []) + [self.value] + (self.right.valuesToList() if self.right else [])
    
    def get(self, key):#normal binary tree get 
        return self.value if self.key == key else (self.right.get(key) if self.key < key else self.left.get(key))
    
    def put(self, key, value, ref):#normal binary tree put
        if self.key == key:
            #always keep furthest point from ref
            self.value = value if value.calcDistance(ref) > self.value.calcDistance(ref) else self.value
        elif self.key < key:
            if self.right is not None:
                self.right.put(key, value, ref)
            else:
                self.right = PointSymbolTable(key, value)
        else:
            if self.left is not None:
                self.left.put(key, value, ref)
            else:
                self.left = PointSymbolTable(key, value)
    
def cross_product(p1,p2,p3):
    return (p2[0] - p1[0])*(p3[1] - p1[1]) - (p3[0] - p1[0])*(p2[1] - p1[1])

def sort_key(start,p1): #sort by -1/tan (angle)
    if p1[1] == start[1]:
        return float('inf') if p1[0] - start[0] < 0 else float('-inf')
    return - (p1[0] - start[0]) / (p1[1] - start[1])                         

def grahamscan(listOfPoints):
    stack=[]
    #find point with lowest y co-ordinate - if y is the same take lowest x
    point0 = listOfPoints[0]
    for current in listOfPoints[1:]:
        if (current[1] < point0[1]) or ((current[1] == point0[1]) and (current[0] < point0[0])):
            point0 = current
    #initialise symbol table with angle key and point value
    symTable = PointSymbolTable(sort_key(point0,listOfPoints[0]),listOfPoints[0])

    #calculate angles then sort by angle using symbol table
    for current in listOfPoints:
        if current != point0:
            symTable.put(sort_key(point0,current),current,point0)
    
    sortedPoints = [point0] + symTable.valuesToList()
    #push first two points onto stack
    stack.append(sortedPoints[0])
    stack.append(sortedPoints[1])
    for i in range(2,len(sortedPoints)):
        while len(stack) > 1 and cross_product(stack[-2],stack[-1],sortedPoints[i]) <= 0:
                #if < 0 then we are turning right so pop the last point from the stack - anticlockwise search
                stack.pop()
        stack.append(sortedPoints[i])
    return stack

Use the cell below for all python code needed to realise the **Chen's** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `chen()` function itself should take as input parameter a list of 2D inputSet (`inputSet`), and return the subset of such inputSet that lie on the convex hull (`outputSet`).

In [3]:

def jarvisMarchModified(hulls):

    # finds the leftmost point from all sub convex hulls
    # in sorting at grahams, each leftmost point is put at 0th index
    bottomLeftPoints = [hull[0] for hull in hulls]
    leftMostHullI = findLeftMostPoint(bottomLeftPoints)


    p = hulls[leftMostHullI][0]
    q = hulls[leftMostHullI][1]
    outputSet = [p]


    # searches each sub convex hull for a more anticlockwise point
    while True:
        for hull in hulls:
            for r in hull:    
                if isBetterOrientation(p, q, r):
                    q = r
        p = q
        if p == outputSet[0]:
            break

        outputSet.append(p)

    return outputSet

def createSubsets(inputSet):
    # size of convex hull approximated at root of total number of points
    m = int(math.sqrt(len(inputSet)))
    while len(inputSet) % m < 3:
        m +=1
    
    subsets = []
    
    for i in range(0, len(inputSet), m):
        subset = inputSet[i: i + m]
        subsets.append(subset)

    return subsets


def chenScan(inputSet):
    # portions points into subsets
    subsets = createSubsets(inputSet)

    # uses grahams to form sub convex hulls
    subConvexHulls = [grahamScan(subset) for subset in subsets]

    # forms final convex hull using jarvis march
    outputSet = jarvisMarchModified(subConvexHulls)
    return outputSet


''' Alternative Chens, faster only for really really large numbers of points 



def createSubsets(inputSet, m):
    subsets = []
    for i in range(0, len(inputSet), m):
        subset = inputSet[i: i + m]
        if len(subset) < 3:
            return []
        subsets.append(subset)

    return subsets

# attempts to solve for a convex hull of size m
# if it fails it then tries for a larger size m (previous m squared each failure)
def chensScan(inputSet):
    t = 0
    outputSet = []

    while not outputSet:
        t += 1
        m = 2 ** 2 ** t

    
        # portions points into subsets
        subsets = createSubsets(inputSet, m)

        #invalid number of subsets, move onto next m
        if not subsets:
            continue
        
        # forms sub convex hulls using grahams
        # then creates a final convex hull using jarvis
        hulls = [grahamScan(subset) for subset in subsets]
        outputSet = jarvisMarchModified(hulls, m)

    return outputSet

'''


Use the cell below to implement the **synthetic data generator** needed by your experimental framework (including any auxiliary data structures and functions you might need - be mindful of code readability and reusability).

In [4]:
import math
import random
import matplotlib.pyplot

def findLeftMostPoint(inputSet):
        leftMostIndex = 0
        for i in range(1, len(inputSet)):
                currentLeftMost = inputSet[leftMostIndex]
                currentPoint = inputSet[i]
                
                # Update to new left most point
                if currentPoint[0] < currentLeftMost[0]:
                        leftMostIndex = i
                
                # If multiple inputSet have same x coord, prioritise largest y
                elif currentPoint[0] == currentLeftMost[0]:
                        if currentPoint[1] > currentLeftMost[1]:
                                leftMostIndex = i
                        
        return leftMostIndex
    
def calcDistance(p, q):
        return abs(p[0] - q[0]) + abs(p[1] - q[1])


def isBetterOrientation(p, q, r):
        crossProduct = (q[0] - p[0]) * (r[1] - p[1]) - (q[1] - p[1]) * (r[0] - p[0])
        
        # Counterclockwise - accept
        if crossProduct < 0:
                return True
        # Collinear - only accept if further away
        elif crossProduct == 0:
                if calcDistance(p, r) > calcDistance(p, q):
                        return True
        
        return False



''' not sure how to order , apologies for the mess '''


class TestDataGenerator():
    """
    A class to represent a synthetic data generator.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__():
        pass


Use the cell below to implement the requested **experimental framework** API.

In [5]:
import timeit
import matplotlib

class ExperimentalFramework():
    """
    A class to represent an experimental framework.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__():
        pass

Use the cell below to illustrate the python code you used to **fully evaluate** the three convex hull algortihms under considerations. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each algorithm using such data, collect information about their execution time, plots results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [6]:
# ADD YOUR TEST CODE HERE 



